Download and transcribe MP3 with wisper3

In [ ]:
!pip install requests torch transformers


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
from urllib.request import urlretrieve
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, AutoModel
from datetime import datetime
import re
import json
import os

In [ ]:
def sanitize_filename(filename):
    # Define a regular expression pattern to match invalid characters
    invalid_chars_pattern = r'[<>:"/\\|?*\x00-\x1F\x7F]'

    # Replace invalid characters with an underscore
    sanitized_filename = re.sub(invalid_chars_pattern, '', filename)

    # Remove leading and trailing whitespaces and dots
    sanitized_filename = sanitized_filename.strip(' .')

    return sanitized_filename

Download audio

In [ ]:
with open("podcast_data.json", "r", encoding='utf-8') as json_file:
    podcast_data_collection = json.load(json_file)


In [ ]:
print(podcast_data_collection[0])

{'title': 'Fremtidens forskning - VidensGab\xa0', 'subtitle': 'Hør videnskabsjournalist Lone Frank fortælle hvordan forskning vil få et kæmpe boost med mønstergenkendelse, autonome laboratorier, kortlægning af alt med E-swiping og identifikation af nye sammenhænge. Det bliver helt vildt hvad der kommer af nye...', 'description': 'Hør videnskabsjournalist Lone Frank fortælle hvordan forskning vil få et kæmpe boost med mønstergenkendelse, autonome laboratorier, kortlægning af alt med E-swiping og identifikation af nye sammenhænge. Det bliver helt vildt hvad der kommer af nye gennembrud. Men er forskerne klar?<br /><br /><b>Værter:</b><br />Liselotte Lyngsø - <a href="https://www.linkedin.com/in/liselotte-lyngs%C3%B8-03a205/" target="_blank" rel="noreferrer noopener">LinkedIn</a><br /><br />Iværksætter og forfatter Lars Tvede<b> </b>- <a href="https://www.linkedin.com/in/lars-tvede/" target="_blank" rel="noreferrer noopener">LinkedIn</a><br /><br /><b>Medvirkende:</b><br />Lone Frank - <a

In [ ]:
current_progress_count = 0
for podcast_data in podcast_data_collection:
    language = podcast_data['language']
    if (language == 'da'):
        language = 'danish'
    if (language == 'en' or language == 'en-US'):
        language = 'english'

    episode_stamp = ""
    if (podcast_data["type"] == "episodic"):
        fmt = "%Y-%m-%d"
        fancy = "%a, %d %b %Y %H:%M:%S %z"
        date_str = podcast_data['pubdate']
        short_date = datetime.strptime(date_str, fancy).strftime("%Y-%m-%d")
        episode_stamp = f" - {podcast_data['podcastnumber']} - {short_date}"
    else:
        episode_stamp = f" - {podcast_data['podcastnumber']}"

    podcast_episode_name = f"{podcast_data['podcasttitle']}{episode_stamp} - {podcast_data['title']}"
    podcast_episode_name = sanitize_filename(podcast_episode_name)

    current_progress_count += 1
    print(
        f"Processing podcast {current_progress_count}/{len(podcast_data_collection)} - {podcast_episode_name}")

    current_podcast_folder = podcast_data['podcasttitle']
    audio_file_dest = f"{podcast_episode_name}.mp3"
    urlretrieve(podcast_data['mp3_url'], audio_file_dest)

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    # Distil is much quicker 18 min pr 50 min audio vs wisper3 40min pr 50 min audio. But only works for English
    model_id = "openai/whisper-large-v3"
    if (language == "english"):
        model_id = "distil-whisper/distil-large-v3"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=25,
        batch_size=16,
        return_timestamps=False,
        torch_dtype=torch_dtype,
        device=device,
    )
    print(f"Transcribing audio text-to-speech using {model_id}")
    if language == "english":
        args = generate_kwargs = {"language": language}
    else:
        args = generate_kwargs = {
            "language": language, "task": "translate"}

    result = pipe(audio_file_dest, generate_kwargs=args)
    print(result["text"])

    with open(f"{podcast_episode_name}.txt", "w", encoding='utf-8') as f:
        f.writelines(result["text"])
    os.remove(audio_file_dest)



Processing podcast 1/30 - Supertrends - 63 - 2024-03-18 - Fremtidens forskning - VidensGab 


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

...

Transcribing audio text-to-speech using openai/whisper-large-v3
 Imagine walking down the dusty streets of Copenhagen as an avatar and visiting a sick little child. A robot will be a university lecturer. This robot will be able to multitask, so you can send an email to it, which it answers digitally, while it is teaching. Then it knows what has been asked about and can then take the initiative. We are starting to see some incredibly intelligent chatbots. They can solve a lot of the tasks we have in the school system. This is Supertrends. The program where we are curiously investigating all the developments that will turn your, my and our lives upside down. Your worth is future researcher Liselotte Lyngsø I am happy to be here. I have been looking forward to making this episode today, because it is about learning technologies. I recently saw an article in Forbes, where it was described that if you are under 5 years old, then you ask your mother and father about 120 things an hour. So yo

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing audio text-to-speech using openai/whisper-large-v3
 And then it takes an hour for this liter of magic to run into my arms. Then I get some white powder that I have to sprinkle on my yogurt every morning. Then I eat my yogurt drink and sprinkle that powder on, and then I become completely ecstatic afterwards, and then people think it's cocaine. These cost-effects in scientific experiments look like they could promote the spread of some forms of force, and they don't help with all the other things they should have helped with. Our very poor children will live through this for 150 years. This is Supertrends, the program where we inquire about all the developments that will turn your, my and our lives upside down. Your worth is a future researcher at Lotte Lyngsø, and every week she is flanked by investor, author and creator behind Supertrends, Lars Tvede eternal life? I do. But when I talk to people about it, there are quite a few who don't want it. But I think it would be fa

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing audio text-to-speech using openai/whisper-large-v3
 We want to have a true copy of what has been when we have all kinds of technologies to make something that is much cooler. There is at least one researcher who claims he has cloned a human, but has stopped trying when the foster only had four cells. I got three clones out of him. So there are actually three small Nintendos now, and they are extremely cute. And they look like each other on a dot, and look like Nintendo on a dot. So it's extremely fun. This is Supertrends. The show where we curiously investigate all the developments that will turn your, my and our lives upside down. Your host is the future researcher Lise Loned and lived a nice life and got some cute children afterwards. And then there is a number of all kinds of crazy and wild science fiction movies that have made us all somehow have a tendency to clone. And for me, all my years as a future researcher, I've always thought, that's the craziest thing about b